<a href="https://colab.research.google.com/github/LucasFDutra/machine_learning_studies/blob/master/machine_learning_studies/Music_Generation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libs


In [1]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
assert len(tf.config.list_physical_devices('GPU')) > 0

     |████████████████████████████████| 2.1MB 9.4MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.1.2-cp36-none-any.whl size=2114585 sha256=73e20f8b3a25885fe221a56963b671f3048b41dc6c155f6ece338171df5341fe
  Stored in directory: /root/.cache/pip/wheels/27/e1/73/5f01c787621d8a3c857f59876c79e304b9b64db9ff5bd61b74
Successfully built mitdeeplearning


# Dataset

Vou pegar os dados com o auxilio da biblioteca do MIT


In [2]:
songs = mdl.lab1.load_training_data()

Found 816 songs in text


In [3]:
songs[0]

'X:2\nT:An Buachaill Dreoite\nZ: id:dc-hornpipe-2\nM:C|\nL:1/8\nK:G Major\nGF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!\nDGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!\nGA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!\nB2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!'

# Criando um vocabulário

Verificar os caracteres únicos

In [4]:
songs_joined = "\n\n".join(songs)

vocab = sorted(set(songs_joined))

print ('Existem ', len(vocab), ' caracteres diferentes no vocabulário')

Existem  83  caracteres diferentes no vocabulário


# Criando um dicionário de dados

Vou pegar todos os carateres presentes no dataset e atribuir um valor numérico para cada um.

ex: a = 0, b = 1, ...

E também criarei um meio de voltarmos, colocando os caracteres em um array

ex: `['a', 'b']`

Assim saberemos que o caractere representado pelo número 0 é o 'a', pois ele ocupa a posição zero no array

In [5]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

print('em char2idx: ', list(char2idx.items())[0])

em char2idx:  ('\n', 0)


# Convertendo o array de letras nos numeros correspondentes do nosso diconário de dados

In [6]:
def vectorize_string(string):
    vectorized_output = np.array([char2idx[char] for char in string])
    return vectorized_output

vectorized_songs = vectorize_string(songs_joined)

vectorized_songs

array([49, 22, 14, ..., 22, 82,  2])

# Definir função para pegar batchs de dados
```python
vectorized_songs = [1,2,3,4]

seq_length = 3

x = [1,2,3]
y = [2,3,4]
```

In [7]:
def get_batch(vectorized_songs, seq_length, batch_size):
    n = len(vectorized_songs) - 1
    idx = np.random.choice(n - seq_length, batch_size)

    input_batch = [vectorized_songs[i : seq_length+i] for i in idx]
    output_batch = [vectorized_songs[i+1 : seq_length+i+1] for i in idx]

    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])

    return x_batch, y_batch

In [8]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

print(x_batch, y_batch)

[[ 1 32 26 27 58]] [[32 26 27 58 82]]


# Construindo o modelo

uma camada `tf.keras.layers.Embedding`

uma camada `tf.keras.layers.LSTM`

uma camada `tf.keras.layers.Dense

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, recurrent_activation='sigmoid', return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (32, None, 256)           21248     
_________________________________________________________________
lstm_1 (LSTM)                (32, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (32, None, 83)            85075     
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [21]:
x, y = get_batch(vectorized_songs=vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
pred

<tf.Tensor: shape=(32, 100, 83), dtype=float32, numpy=
array([[[-1.8323732e-03,  4.1002324e-03,  3.3811606e-03, ...,
          7.5380877e-03,  1.0129531e-02, -2.4274306e-04],
        [-3.5257824e-03, -2.6582791e-03,  1.6801446e-03, ...,
         -3.2416609e-04,  6.7239432e-03, -2.6758511e-03],
        [-6.6769756e-03, -3.5394996e-04,  2.0995210e-03, ...,
         -7.2361622e-03,  2.0319573e-03, -2.4986470e-03],
        ...,
        [ 6.5698484e-03, -5.3877803e-04, -4.1704131e-03, ...,
         -3.6411888e-03, -1.3557812e-03, -1.2198782e-02],
        [ 1.0665262e-02,  7.1084395e-04, -2.4555698e-03, ...,
         -8.4243203e-03, -2.1591131e-04, -5.6810174e-03],
        [ 9.8171365e-03,  1.1887031e-03, -1.0693056e-04, ...,
         -3.7951616e-03,  3.4708644e-03, -5.3682453e-03]],

       [[-3.3320910e-03, -5.2235401e-03, -8.5501643e-03, ...,
         -8.2426029e-04, -3.5396740e-03, -7.7688233e-03],
        [-6.8058679e-03, -1.7128958e-03, -6.7624385e-03, ...,
         -6.5903431e-03, -6.